In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict()

In [ ]:
hg = OnePiece.gene.read_hgnc()

pr_ = hg[!, "locus_group"] .== "protein-coding gene"

pr_ = sort(unique(skipmissing(hg[pr_, "symbol"])))

## Ensembl

In [ ]:
hu_mo = OnePiece.gene.map_with_mouse(ho = "human_to_mouse")

me_ge_["Ensembl"] = [pr for pr in pr_ if !haskey(hu_mo, pr)]

## Mouse Genome Informatics

In [ ]:
mg = OnePiece.table.read(joinpath(PAI, "hom_mousehumansequence.rpt.txt.gz"))

hu_ = mg[!, "Common Organism Name"] .== "human"

mg[hu_, "Symbol"] .= OnePiece.gene.rename(mg[hu_, "Symbol"])[1]

hu_mo = Dict()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    hu_ = da[!, "Common Organism Name"] .== "human"

    for hu in sy_[hu_], mo in sy_[.!hu_]

        hu_mo[hu] = mo

    end

end

println(length(hu_mo))

me_ge_["MGI"] = [pr for pr in pr_ if !haskey(hu_mo, pr)]

## 2011 De Novo Origin of Human Protein-Coding Genes

In [ ]:
tr_ = readlines(joinpath(PAI, "de_novo_origin_of_human_protein_coding_genes", "genes.txt"))

me_ge_["2011 paper"] = OnePiece.gene.rename(["CLLU1", "c22orf45", "DNAH10OS", tr_...])[1]

## 2019 Genes with human-specific features are primarily involved with brain, immune and metabolic evolution

In [ ]:
da = OnePiece.table.read(
    joinpath(
        PAI,
        "genes_with_human_specific_features_are_primarily_involved_with_brain,_immune_and_metabolic_evolution",
        "12859_2019_2886_MOESM2_ESM.xlsx",
    ),
    xl = "HumanSpecific Genes",
)

me_ge_["2019 paper"] = OnePiece.gene.rename(string.(da[!, "Gene Name"]))[1]

## Write

In [ ]:
write_gene_by_method(PAO, "human", me_ge_)